In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import fbeta_score, recall_score, precision_score, average_precision_score
from sklearn.metrics import precision_recall_curve, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
import seaborn as sns
from collections import Counter
from sklearn.metrics import make_scorer
from xgboost import XGBClassifier
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cd "/content/drive/My Drive/Case_Competition/Humana_2020/Data"

/content/drive/My Drive/Case_Competition/Humana_2020/Data


In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(max_mem_size='16G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpzjg1ca5v
  JVM stdout: /tmp/tmpzjg1ca5v/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpzjg1ca5v/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_unknownUser_kshmeo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
train  = h2o.import_file('/content/drive/My Drive/Case_Competition/Humana_2020/pca_smote_sampled.csv')
test = h2o.import_file('/content/drive/My Drive/Case_Competition/Humana_2020/pca_unseen_data.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
# Identify predictors and response
x = train.columns
y = "transportation_issues"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_runtime_secs=600, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20201011_150054,0.905322,0.389469,0.896823,0.17663,0.350654,0.122958
StackedEnsemble_AllModels_AutoML_20201011_150054,0.905322,0.389469,0.896823,0.17663,0.350654,0.122958
DRF_1_AutoML_20201011_150054,0.888374,0.454929,0.876421,0.198145,0.375211,0.140784
XRT_1_AutoML_20201011_150054,0.882769,0.461085,0.871428,0.198217,0.379885,0.144313
XGBoost_2_AutoML_20201011_150054,0.806345,0.544149,0.785377,0.281144,0.426623,0.182007
XGBoost_grid__1_AutoML_20201011_150054_model_1,0.803505,0.541178,0.777743,0.278465,0.425978,0.181457
GBM_grid__1_AutoML_20201011_150054_model_2,0.800791,0.555441,0.774444,0.28667,0.431901,0.186539
GBM_4_AutoML_20201011_150054,0.792535,0.582345,0.766758,0.295428,0.444252,0.19736
GBM_5_AutoML_20201011_150054,0.771242,0.601559,0.749345,0.324896,0.453937,0.206059
XGBoost_1_AutoML_20201011_150054,0.770134,0.579835,0.746264,0.315658,0.444327,0.197426


In [ ]:
preds = aml.leader.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [ ]:
preds

predict,p0,p1
0,0.922922,0.0770784
1,0.542715,0.457285
1,0.364601,0.635399
0,0.813493,0.186507
1,0.501189,0.498811
0,0.68906,0.31094
0,0.953141,0.046859
0,0.904302,0.0956983
0,0.95927,0.04073
0,0.977633,0.0223667


In [ ]:
perf = aml.leader.model_performance(test)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.17971362233004748
RMSE: 0.4239264350451001
LogLoss: 0.5637444170849557
Null degrees of freedom: 20871
Residual degrees of freedom: 20869
Null deviance: 28934.7359052934
Residual deviance: 23532.946946794393
AIC: 23538.946946794393
AUC: 0.6142453655160185
AUCPR: 0.21893098995109248
Gini: 0.228490731032037

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.18873942270186322: 


,,0,1,Error,Rate
0,0,10778.0,6976.0,0.3929,(6976.0/17754.0)
1,1,1361.0,1757.0,0.4365,(1361.0/3118.0)
2,Total,12139.0,8733.0,0.3994,(8337.0/20872.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.188739,0.296515,281.0
1,max f2,0.023516,0.470701,387.0
2,max f0point5,0.535764,0.256446,142.0
3,max accuracy,0.975729,0.850565,0.0
4,max precision,0.975729,0.400000,0.0
5,max recall,0.010773,1.000000,399.0
6,max specificity,0.975729,0.999831,0.0
7,max absolute_mcc,0.382249,0.130889,196.0
8,max min_per_class_accuracy,0.171244,0.583708,290.0
9,max mean_per_class_accuracy,0.188739,0.585288,281.0



Gains/Lift Table: Avg response rate: 14.94 %, avg score: 25.23 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010013,0.937127,2.242021,2.242021,0.334928,0.952382,0.334928,0.952382,0.022450,0.022450,124.202117,124.202117,0.014621
1,2,0.020027,0.911254,1.889704,2.065862,0.282297,0.923269,0.308612,0.937825,0.018922,0.041373,88.970356,106.586236,0.025095
2,3,0.030040,0.888339,1.665501,1.932409,0.248804,0.899576,0.288676,0.925075,0.016677,0.058050,66.550144,93.240872,0.032929
3,4,0.040006,0.865275,2.156252,1.988168,0.322115,0.876470,0.297006,0.912968,0.021488,0.079538,115.625154,98.816837,0.046475
4,5,0.050019,0.840844,2.081877,2.006928,0.311005,0.853516,0.299808,0.901066,0.020847,0.100385,108.187680,100.692801,0.059211
5,6,0.100038,0.717591,1.711980,1.859454,0.255747,0.780068,0.277778,0.840567,0.085632,0.186017,71.198012,85.945407,0.101078
6,7,0.150010,0.592908,1.360628,1.693285,0.203260,0.652271,0.252954,0.777842,0.067992,0.254009,36.062833,69.328503,0.122264
7,8,0.200029,0.486523,1.314442,1.598552,0.196360,0.538719,0.238802,0.718047,0.065747,0.319756,31.444167,59.855151,0.140754
8,9,0.300019,0.313651,1.189979,1.462383,0.177767,0.393755,0.218461,0.609967,0.118987,0.438743,18.997932,46.238253,0.163087
9,10,0.400010,0.204716,1.042435,1.357408,0.155726,0.256179,0.202779,0.521530,0.104233,0.542976,4.243472,35.740815,0.168075


In [ ]:
train  = h2o.import_file('/content/drive/My Drive/Case_Competition/Humana_2020/pca_seen_data.csv')
test = h2o.import_file('/content/drive/My Drive/Case_Competition/Humana_2020/pca_unseen_data.csv')
final_test = h2o.import_file('/content/drive/My Drive/Case_Competition/Humana_2020/pca_test.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
# Identify predictors and response
x = train.columns
y = "transportation_issues"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_runtime_secs=1800, seed=1, balance_classes=True, keep_cross_validation_fold_assignment=True)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20201011_164413,0.678388,0.390027,0.261955,0.370938,0.343072,0.117698
StackedEnsemble_BestOfFamily_AutoML_20201011_164413,0.677083,0.390423,0.259415,0.370745,0.343267,0.117832
DeepLearning_grid__3_AutoML_20201011_164413_model_1,0.672398,0.391619,0.253898,0.369875,0.343789,0.118191
GBM_grid__1_AutoML_20201011_164413_model_1,0.671764,0.391775,0.253548,0.372044,0.343834,0.118222
GBM_grid__1_AutoML_20201011_164413_model_10,0.671068,0.391642,0.252427,0.372751,0.343769,0.118177
DeepLearning_grid__2_AutoML_20201011_164413_model_1,0.671007,0.392587,0.251988,0.370776,0.344322,0.118557
GBM_grid__1_AutoML_20201011_164413_model_7,0.670945,0.391643,0.252699,0.374391,0.343762,0.118172
GBM_grid__1_AutoML_20201011_164413_model_8,0.670219,0.39192,0.251195,0.372344,0.343904,0.11827
XGBoost_grid__1_AutoML_20201011_164413_model_11,0.669835,0.392264,0.25115,0.37091,0.344087,0.118396
GLM_1_AutoML_20201011_164413,0.669118,0.392546,0.249075,0.371745,0.344239,0.118501


In [ ]:
preds = aml.leader.predict(test)
preds

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1
0,0.888166,0.111834
1,0.806768,0.193232
1,0.762365,0.237635
0,0.890248,0.109752
0,0.885301,0.114699
0,0.884034,0.115966
0,0.872901,0.127099
1,0.763901,0.236099
0,0.890546,0.109454
0,0.906943,0.093057


In [ ]:
data_as_df = preds.as_data_frame()
data_as_df

,predict,p0,p1
0,0,0.888166,0.111834
1,1,0.806768,0.193232
2,1,0.762365,0.237635
3,0,0.890248,0.109752
4,0,0.885301,0.114699
...,...,...,...
20867,1,0.797834,0.202166
20868,0,0.891687,0.108313
20869,0,0.885335,0.114665
20870,0,0.917637,0.082363


In [ ]:
data_as_df.to_csv(r'/content/drive/My Drive/Case_Competition/Humana_2020/unseen_test_probab.csv', header=True)

In [ ]:
data_as_df['predict'].value_counts()

0    12354
1     5327
Name: predict, dtype: int64

In [ ]:
perf = aml.leader.model_performance(test)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.12033105603751652
RMSE: 0.34688767063347253
LogLoss: 0.39751829917962
Null degrees of freedom: 20871
Residual degrees of freedom: 20855
Null deviance: 17603.805268614316
Residual deviance: 16594.003880954057
AIC: 16628.003880954057
AUC: 0.6713650089098082
AUCPR: 0.2688379039999492
Gini: 0.34273001781961643

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.15660203410414097: 


,,0,1,Error,Rate
0,0,12825.0,4929.0,0.2776,(4929.0/17754.0)
1,1,1424.0,1694.0,0.4567,(1424.0/3118.0)
2,Total,14249.0,6623.0,0.3044,(6353.0/20872.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.156602,0.347808,248.0
1,max f2,0.108433,0.487708,320.0
2,max f0point5,0.216516,0.310723,178.0
3,max accuracy,0.612854,0.850757,2.0
4,max precision,0.710393,1.000000,0.0
5,max recall,0.064481,1.000000,397.0
6,max specificity,0.710393,1.000000,0.0
7,max absolute_mcc,0.181496,0.203938,216.0
8,max min_per_class_accuracy,0.136075,0.629886,276.0
9,max mean_per_class_accuracy,0.153131,0.632910,252.0



Gains/Lift Table: Avg response rate: 14.94 %, avg score: 14.99 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010013,0.403492,2.690425,2.690425,0.401914,0.453055,0.401914,0.453055,0.026940,0.026940,169.042540,169.042540,0.019900
1,2,0.020027,0.370682,2.658397,2.674411,0.397129,0.385396,0.399522,0.419225,0.026620,0.053560,165.839653,167.441097,0.039422
2,3,0.030040,0.349666,2.466223,2.605015,0.368421,0.359384,0.389155,0.399278,0.024695,0.078255,146.622329,160.501507,0.056683
3,4,0.040006,0.332265,2.317166,2.533311,0.346154,0.340846,0.378443,0.384723,0.023092,0.101347,131.716584,153.331131,0.072114
4,5,0.050019,0.318303,2.081877,2.442938,0.311005,0.324763,0.364943,0.372719,0.020847,0.122194,108.187680,144.293793,0.084850
5,6,0.100038,0.267801,1.981280,2.212109,0.295977,0.291495,0.330460,0.332107,0.099102,0.221296,98.128037,121.210915,0.142553
6,7,0.150010,0.227646,1.758548,2.061019,0.262704,0.247595,0.307889,0.303954,0.087877,0.309173,75.854793,106.101865,0.187116
7,8,0.200029,0.199698,1.660685,1.960911,0.248084,0.213236,0.292934,0.281269,0.083066,0.392239,66.068484,96.091122,0.225966
8,9,0.300019,0.161029,1.260544,1.727493,0.188309,0.178574,0.258065,0.247043,0.126042,0.518281,26.054414,72.749281,0.256593
9,10,0.400010,0.137406,1.029605,1.553042,0.153809,0.148406,0.232004,0.222387,0.102951,0.621232,2.960475,55.304169,0.260074
